In [137]:
!pip install pyaudio
!pip install sounddevice
!pip install speechrecognition
!pip install scipy

In [138]:
!pip install spacy
#python -m spacy download en_core_web_sm


In [160]:
import nltk
import re
import pandas as pd

# Sample transaction data
'''transactions = [
    "Bought 50rs of bananas",
    "Rs. 100 at Starbucks",
    "Paid 200rs for electricity bill",
    "Taxi fare of 150rs",
    "Movie ticket for 300rs",
    "Paid 75 bucks at the cafe",
    "Grocery shopping for 120 rupees",
    "Spent 500 rs on a new phone",
    "Cost 25 rupee for the book",
    "Spent 50.5 rs on fruits",
    "I spent Rupees 100 on water",
    "I spent 200 rupees on groceries",
    "I bought a movie ticket for 500 rupees",
    "Spent 300 on bus fare",
    "Paid 1000 rupees for a doctor visit",
    "Bought new furniture for 1500 rupees",
    "Spent 600 on books",
    "Paid 700 for electricity bill",
    "Spent 1200 rupees on a concert",
    "I spent 250 on fruits",
    "Paid 400 for a taxi ride",
    "I visited the doctor and spent 1000rs"
]'''

transactions=["Bought 50rs of bananas", "Rs. 100 at Starbucks", "Paid 200rs for electricity bill", "Taxi fare of 150rs", "Movie ticket for 300rs", "Paid 75 bucks at the cafe", "Grocery shopping for 120 rupees", "Spent 500 rs on a new phone", "Cost 25 rupee for the book", "Spent 50.5 rs on fruits", "I spent Rupees 100 on water", "I spent 200 rupees on groceries", "I bought a movie ticket for 500 rupees", "Spent 300 on bus fare", "Paid 1000 rupees for a doctor visit", "Bought new furniture for 1500 rupees", "Spent 600 on books", "Paid 700 for electricity bill", "Spent 1200 rupees on a concert", "I spent 250 on fruits", "Paid 400 for a taxi ride", "I visited the doctor and spent 1000rs", "Spent 220 rupees on transportation", "Spent 230 rupees on entertainment", "Spent 240 rupees on household", "Spent 250 rupees on shopping", "Spent 260 rupees on health", "Spent 270 rupees on education", "Spent 280 rupees on gift", "Spent 290 rupees on others", "Spent 300 rupees on food", "Spent 310 rupees on social_life", "Spent 3460 rupees on health", "Spent 3470 rupees on education", "Spent 3480 rupees on gift", "Spent 3490 rupees on others", "Spent 450 rupees on dinner with friends.", "Bought a souvenir for 200 rupees.", "Paid 75 rs for a postcard.", "Spent 300 rupees on a photo frame.", "Bought a wallet for 500 rupees.", "Spent 120 rupees on face wash.", "Paid 40 rs for a bookmark.", "Bought a cup of coffee for 150 rupees.", "Paid 350 rupees for a doctor's appointment.", "I spent 500 on a nice lunch.", "Bought new headphones for 1000 rupees.", "Cost 75 rupees for the magazine.", "Paid 200 for parking fees.", "Spent 60 rupees on a bottle of water.", "Taxi fare of 200 rupees.", "Bought groceries worth 500 rupees.", "Paid 100 rupees for movie snacks.", "Spent 120 rupees on stationery.", "Paid 300 rs for a haircut.", "Spent 200 rupees on a train ticket.", "Bought snacks for 100 rupees.", "Paid 75 rs for coffee.", "Spent 50 rupees on a juice.", "Paid 600 rs for a fitness class.", "Bought clothes worth 1000 rupees.", "Spent 200 rupees on a bus fare.", "Cost 150 rupees at the bakery.", "Spent 40 rupees on gum.", "Paid 1200 rs for the dentist.", "Bought a new bag for 700 rupees.", "Spent 100 rs on phone recharge.", "Paid 300 rupees for flowers.", "Spent 500 rs on a family dinner.", "Paid 250 for a museum ticket.", "Bought books worth 350 rupees.", "Spent 80 rupees on tea.", "Cost 900 rupees at the optician.", "Paid 1000 rs for sports equipment.", "Spent 500 rupees on clothes.", "Taxi fare of 350 rs.", "Spent 50 rupees on stationery.", "Paid 60 rs for an ice cream.", "Spent 2000 rupees on a new pair of shoes.", "Bought a cake for 500 rupees.", "Paid 1000 rs for the hotel stay.", "Spent 150 rupees on a meal.", "Bought tickets worth 800 rupees.", "Spent 350 on a concert ticket.", "Paid 400 rupees for medical supplies.", "Spent 100 rs on a movie rental.", "Cost 75 rs for entry fees.", "Bought a hat for 250 rupees.", "Spent 180 rupees on phone accessories.", "Paid 500 rs for a cooking class.", "Spent 120 rupees on a gift.", "Bought fruit worth 75 rupees.", "Spent 650 rs on a dress.", "Paid 2500 rupees on an online course.", "Spent 300 rupees on lunch.", "Taxi fare of 100 rs.", "Spent 200 rupees on a pedicure.", "Paid 350 rs for a game.", "Bought groceries for 250 rupees.", "Paid 125 rs at the coffee shop.", "Spent 90 rupees on a snack.", "Bought veggies worth 200 rupees.", "Spent 1200 rs on a fitness tracker.", "Paid 45 rs for a candy bar.", "Taxi fare of 150 rs.", "Spent 300 rupees on groceries.", "Bought a magazine for 50 rs.", "Paid 75 rupees for the library fee.", "Spent 250 rupees on a movie ticket.", "Spent 100 rupees on a cold drink.", "Paid 700 rupees for car service.", "Bought accessories for 400 rs.", "Spent 60 rupees on popcorn.", "Paid 500 rupees for a phone case.", "Bought a t-shirt for 300 rupees.", "Paid 80 rs for a notebook.", "Spent 150 rupees on stationery.", "Paid 450 rs at the dentist.", "Bought flowers worth 100 rupees.", "Spent 1200 rupees on groceries.", "Taxi fare of 90 rs.", "Spent 200 rs on a cab ride.", "Paid 120 rupees at the coffee shop.", "Bought a scarf for 200 rupees.", "Spent 75 rs on street food.", "Paid 300 rupees for gym membership.", "Spent 2500 rs on a hotel room.", "Bought an umbrella for 150 rupees.", "Spent 100 rs on a drink.", "Paid 500 rs for a city tour.", "Bought a novel for 350 rupees.", "Spent 120 rupees on juice.", "Paid 400 rupees for car wash.", "Bought a jacket worth 800 rs.", "Spent 150 rupees on a meal.", "Paid 90 rupees for a newspaper.", "Spent 200 rupees on dinner.", "Bought a bag for 1000 rupees.", "Paid 70 rupees for a bookmark.", "Spent 350 rs on laundry.", "Bought cosmetics for 200 rupees.", "Paid 50 rupees for a pen.", "Spent 1200 rupees on a concert ticket.", "Spent 180 rupees on stationery.", "Bought a sandwich for 50 rupees", "Paid 200 rupees for gym fees", "Spent 150 rupees on a coffee", "Bought a book for 300 rupees", "Paid 100 rupees for snacks", "Spent 400 rupees on a shirt", "Taxi fare of 120 rupees", "Bought a pen for 20 rupees", "Spent 500 rupees on a birthday gift", "Paid 60 rupees for a bottle of water", "Spent 150 rupees on dinner", "Bought groceries for 700 rupees", "Spent 80 rupees on a magazine", "Paid 1000 rupees for a health check-up", "Bought a concert ticket for 900 rupees", "Spent 50 rupees on stationery", "Paid 75 rupees for coffee at Starbucks", "Spent 60 rupees on a snack", "Bought a scarf for 200 rupees", "Paid 400 rupees for lunch with friends", "Spent 30 rupees on gum", "Bought a new water bottle for 100 rupees", "Spent 350 rupees on school supplies", "Paid 300 rupees for a taxi ride", "Bought a phone case for 150 rupees", "Spent 200 rupees on a bus ticket", "Paid 80 rupees for a notepad", "Bought a pack of biscuits for 50 rupees", "Spent 120 rupees on movie snacks", "Paid 600 rupees for a cooking class", "Spent 400 rupees on a family meal", "Bought a new wallet for 250 rupees", "Paid 45 rupees for a soda", "Spent 350 rupees on a museum ticket", "Bought a fruit basket for 150 rupees", "Spent 220 rupees on a taxi ride", "Paid 75 rupees for an ice cream", "Spent 100 rupees on chocolates", "Bought a novel for 250 rupees", "Paid 2000 rupees for a weekend stay", "Spent 90 rupees on a fruit juice", "Bought groceries for 600 rupees", "Spent 800 rupees on a new shirt", "Paid 70 rupees for a tea", "Bought a lunchbox for 200 rupees", "Spent 50 rupees on a cold drink", "Paid 400 rupees for flowers", "Bought a pair of sunglasses for 300 rupees", "Spent 75 rupees on parking fees", "Paid 100 rupees for movie rental", "Bought a coffee for 120 rupees", "Spent 600 rupees on groceries", "Paid 300 rupees for car wash", "Bought a cake for 250 rupees", "Spent 400 rupees on a fitness tracker", "Paid 1500 rupees for gym membership", "Bought a novel for 350 rupees", "Spent 100 rupees on a drink", "Paid 50 rupees for candy", "Bought a jacket for 700 rupees", "Spent 500 rupees on a family dinner", 
"Paid 120 rupees for street food", "Bought cosmetics for 300 rupees", "Spent 150 rupees on a juice", "Paid 700 rupees for doctor’s appointment", "Bought a keychain for 50 rupees", "Spent 60 rupees on popcorn", "Paid 300 rupees for a movie", "Bought a new pair of shoes for 1200 rupees", "Spent 500 rupees on groceries", "Paid 100 rupees for a bus pass", "Bought a souvenir for 100 rupees", "Spent 200 rupees on dinner with friends", "Paid 90 rupees for a newspaper", "Bought a bottle of water for 20 rupees", "Spent 300 rupees on phone accessories", "Paid 600 rupees for a cooking course", "Bought a birthday gift for 400 rupees", "Spent 500 rupees on a backpack", "Paid 200 rupees for a train ticket", "Bought a magazine for 60 rupees", "Spent 40 rupees on stationery", "Paid 100 rupees for snacks", "Bought groceries worth 750 rupees", "Spent 1500 rupees on a weekend getaway", "Paid 250 rupees for entry fees", "Bought an umbrella for 300 rupees", "Spent 1000 rupees on a gym subscription", "Paid 30 rupees for a bookmark", "Bought a new bag for 900 rupees", "Spent 600 rupees on clothes", "Paid 200 rupees for a manicure", "Bought a movie ticket for 400 rupees", "Spent 60 rupees on coffee", "Paid 150 rupees for a magazine", "Bought a birthday card for 20 rupees", "Spent 300 rupees on a scarf", "Paid 80 rupees for a cold drink", "Bought a fitness band for 700 rupees", "Spent 50 rupees on a snack" ]

# Define the improved extraction function
def extract_amount_item(transaction):
    # Tokenize the transaction text
    tokens = nltk.word_tokenize(transaction)
    
    # POS tagging
    pos_tags = nltk.pos_tag(tokens)
    
    # Compile a list of common currency words to exclude from item extraction
    currency_terms = {"rs", "rupees", "bucks", "dollars", "pounds", "cost","rupee", "lakh", "crore", "million"}
    
    # Extract the amount using a regex pattern and only keep the numeric part
    amount_pattern = re.compile(r'\b(\d+(?:\.\d+)?)\s*(?:rs|rupees|bucks|dollars|pounds)?', re.IGNORECASE)
    amount_match = amount_pattern.search(transaction)
    amount = amount_match.group(1) if amount_match else "Unknown"
    
    # Extract nouns as potential items, filtering out currency terms
    item_tokens = []
    for word, tag in pos_tags:
        # Collect nouns (NN, NNS) that are not currency terms
        if tag in ['NN', 'NNS'] and word.lower() not in currency_terms:
            item_tokens.append(word)
    
    # Join tokens to form the item name, if available
    item = " ".join(item_tokens) if item_tokens else "Unknown"
    
    return amount.strip(), item.strip()

# Apply the refined extraction to all transactions
extracted_data = [extract_amount_item(transaction) for transaction in transactions]

# Create a DataFrame for better visualization
df = pd.DataFrame(extracted_data, columns=['Amount', 'Item'])

# Print the result
print(df)

    Amount              Item
0       50           bananas
1      100         Starbucks
2      200  electricity bill
3      150              fare
4      300            ticket
..     ...               ...
239     20              card
240    300             scarf
241     80             drink
242    700      fitness band
243     50             snack

[244 rows x 2 columns]


In [140]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [141]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer


# Define expanded categories with additional relevant synsets in WordNet
hypernym_categories = {
    'food': [
        wn.synset('food.n.01'), wn.synset('meal.n.01'), wn.synset('snack.n.01'), wn.synset('drink.n.01'),
        wn.synset('dish.n.01'), wn.synset('fruit.n.01'), wn.synset('vegetable.n.01'), wn.synset('dessert.n.01'),
        wn.synset('grocery.n.01'), wn.synset('produce.n.01'), wn.synset('beverage.n.01'), wn.synset('cooking.n.01'),
        wn.synset('eating.n.01'), wn.synset('cuisine.n.01'), wn.synset('dairy_product.n.01'), wn.synset('meat.n.01'),
        wn.synset('seafood.n.01'), wn.synset('spice.n.01'), wn.synset('ingredient.n.01')
    ],
    'social_life': [
        wn.synset('recreation.n.01'), wn.synset('celebration.n.01'), wn.synset('outing.n.01'), wn.synset('party.n.01'),
        wn.synset('dancing.n.01'), wn.synset('social_event.n.01'), wn.synset('entertainment.n.01'), wn.synset('concert.n.01'),
        wn.synset('movie.n.01'), wn.synset('gathering.n.01'), wn.synset('festival.n.01'), wn.synset('show.n.01'),
        wn.synset('nightclub.n.01'), wn.synset('bar.n.01'), wn.synset('pub.n.01'), wn.synset('celebration.n.01')
    ],
    'transportation': [
        wn.synset('vehicle.n.01'), wn.synset('transportation.n.01'), wn.synset('public_transport.n.01'), wn.synset('taxi.n.01'),
        wn.synset('airplane.n.01'), wn.synset('car.n.01'), wn.synset('train.n.01'), wn.synset('bus.n.01'),
        wn.synset('fare.n.01'), wn.synset('subway.n.01'), wn.synset('railway.n.01'), wn.synset('transport.n.01'),
        wn.synset('commute.n.01'), wn.synset('travel.n.01'), wn.synset('motor_vehicle.n.01'), wn.synset('bicycle.n.01'),
        wn.synset('ticket.n.01'), wn.synset('road.n.01')
    ],
    'entertainment': [
        wn.synset('culture.n.01'), wn.synset('art.n.01'), wn.synset('entertainment.n.01'), wn.synset('concert.n.01'),
        wn.synset('museum.n.01'), wn.synset('movie.n.01'), wn.synset('game.n.01'), wn.synset('sport.n.01'),
        wn.synset('amusement.n.01'), wn.synset('theater.n.01'), wn.synset('exhibition.n.01'), wn.synset('show.n.01'),
        wn.synset('play.n.01'), wn.synset('performance.n.01'), wn.synset('event.n.01'), wn.synset('hobby.n.01')
    ],
    'household': [
        wn.synset('household.n.01'), wn.synset('furniture.n.01'), wn.synset('appliance.n.01'), wn.synset('utility.n.01'),
        wn.synset('cleaning.n.01'), wn.synset('kitchenware.n.01'), wn.synset('decoration.n.01'), wn.synset('bedding.n.01'),
        wn.synset('laundry.n.01'), wn.synset('repair.n.01'), wn.synset('gardening.n.01'), wn.synset('maintenance.n.01'),
        wn.synset('utensil.n.01'), wn.synset('fixture.n.01'), wn.synset('housework.n.01')
    ],
    'shopping': [
        wn.synset('clothing.n.01'), wn.synset('footwear.n.01'), wn.synset('accessory.n.01'), wn.synset('outerwear.n.01'),
        wn.synset('toiletry.n.01'), wn.synset('cosmetic.n.01'), wn.synset('jewelry.n.01'), wn.synset('apparel.n.01'),
        wn.synset('bag.n.01'), wn.synset('watch.n.01'), wn.synset('fashion.n.01'), wn.synset('retail.n.01'),
        wn.synset('boutique.n.01'), wn.synset('department_store.n.01'), wn.synset('shopping.n.01')
    ],
    'health': [
        wn.synset('health.n.01'), wn.synset('medicine.n.01'), wn.synset('therapy.n.01'), wn.synset('fitness.n.01'),
        wn.synset('exercise.n.01'), wn.synset('training.n.01'), wn.synset('meditation.n.01'), wn.synset('nutrition.n.01'),
        wn.synset('doctor.n.01'), wn.synset('hospital.n.01'), wn.synset('wellness.n.01'), wn.synset('pharmacy.n.01'),
        wn.synset('clinic.n.01'), wn.synset('dentist.n.01'), wn.synset('treatment.n.01'), wn.synset('nurse.n.01'),
        wn.synset('diet.n.01')
    ],
    'education': [
        wn.synset('education.n.01'), wn.synset('schooling.n.01'), wn.synset('book.n.01'), wn.synset('course.n.01'),
        wn.synset('lecture.n.01'), wn.synset('research.n.01'), wn.synset('workshop.n.01'), wn.synset('library.n.01'),
        wn.synset('stationery.n.01'), wn.synset('tuition.n.01'), wn.synset('student.n.01'), wn.synset('class.n.01'),
        wn.synset('teacher.n.01'), wn.synset('exam.n.01'), wn.synset('assignment.n.01'), wn.synset('university.n.01')
    ],
    'gift': [
        wn.synset('gift.n.01'), wn.synset('present.n.01'), wn.synset('souvenir.n.01'), wn.synset('donation.n.01'),
        wn.synset('offering.n.01'), wn.synset('keepsake.n.01'), wn.synset('award.n.01'), wn.synset('memento.n.01'),
        wn.synset('prize.n.01'), wn.synset('trophy.n.01'), wn.synset('celebration.n.01'), wn.synset('token.n.01')
    ],
    'others': [
        wn.synset('artifact.n.01'), wn.synset('object.n.01'), wn.synset('thing.n.01'), wn.synset('item.n.01'),
        wn.synset('material.n.01'), wn.synset('equipment.n.01'), wn.synset('supply.n.01'), wn.synset('instrumentality.n.01'),
        wn.synset('device.n.01'), wn.synset('commodity.n.01'), wn.synset('resource.n.01'), wn.synset('tool.n.01')
    ]
}

# Flatten synsets for each category to enable direct comparison
flattened_hypernym_categories = {cat: synsets for cat, synsets in hypernym_categories.items()}

lemmatizer = WordNetLemmatizer()

direct_mappings = {
    'bananas': 'food', 'starbucks': 'food', 'electricity bill': 'household', 'fare': 'transportation', 
    'ticket': 'entertainment', 'cafe': 'food', 'shopping': 'shopping', 'phone': 'entertainment', 
    'rupee book': 'education', 'water': 'food', 'groceries': 'food', 'movie ticket': 'entertainment',
    'bus fare': 'transportation', 'doctor visit': 'health', 'furniture': 'household', 'books': 'education', 
    'electricity': 'household', 'concert': 'social_life', 'taxi ride': 'transportation'
}

# Helper function to check if any synset in a hypernym path matches the target synsets for a category
def is_hypernym_in_path(synset, target_hypernyms):
    hypernym_paths = synset.hypernym_paths()  # Get all hypernym paths
    for path in hypernym_paths:
        for ancestor in path:
            if ancestor in target_hypernyms:
                return True
    return False

# Get category by hypernym path and semantic similarity
def get_category(item):
    # Take only the first word if the item has multiple words
    first_word = item.split()[0]
    
    # Normalize and lemmatize the first word
    lemmatized_item = lemmatizer.lemmatize(first_word.lower())
    
    # Check direct mappings first
    if lemmatized_item in direct_mappings:
        return direct_mappings[lemmatized_item]
    
    # Get synsets for the item as a noun
    item_synsets = wn.synsets(lemmatized_item, pos=wn.NOUN)
    for synset in item_synsets:
        # Check if any synset in the item's hypernym path matches a category
        for category, target_hypernyms in hypernym_categories.items():
            if is_hypernym_in_path(synset, target_hypernyms):
                return category

    # As a last resort, use semantic similarity to find the closest category
    best_category = 'others'
    max_similarity = 0
    for synset in item_synsets:
        for category, target_hypernyms in hypernym_categories.items():
            for target_synset in target_hypernyms:
                similarity = synset.path_similarity(target_synset)
                if similarity and similarity > max_similarity:
                    max_similarity = similarity
                    best_category = category
    return best_category


# Sample dataset
spending_data = [
    {'item': 'fries', 'amount': 100},
    {'item': 'taxi', 'amount': 50},
    {'item': 'concert', 'amount': 200},
    {'item': 'jacket', 'amount': 150},
    {'item': 'phone', 'amount': 300},
    {'item': 'sofa', 'amount': 500},
    {'item': 'medicine', 'amount': 100},
    {'item': 'shampoo', 'amount': 30},
    {'item': 'textbook', 'amount': 200},
    {'item': 'beer', 'amount': 60},
    {'item': 'electricity', 'amount': 80},
    {'item': 'movie', 'amount': 1200},
    {'item': 'yoga', 'amount': 300},
    {'item': 'coffee', 'amount': 50},
]

# Categorize each item
for entry in spending_data:
    item = entry['item']
    category = get_category(item)
    print(f"Item: {item}, Category: {category}")


Item: fries, Category: others
Item: taxi, Category: transportation
Item: concert, Category: social_life
Item: jacket, Category: shopping
Item: phone, Category: entertainment
Item: sofa, Category: household
Item: medicine, Category: health
Item: shampoo, Category: others
Item: textbook, Category: education
Item: beer, Category: food
Item: electricity, Category: household
Item: movie, Category: social_life
Item: yoga, Category: entertainment
Item: coffee, Category: food


In [161]:
df['Category'] = df['Item'].apply(get_category)
df

# Save the DataFrame to an Excel file
df.to_excel('path_to_your_file2.xlsx', index=False)


In [143]:
import pyaudio
import numpy as np
import speech_recognition as sr
import time
import io
import re
import pandas as pd
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")


RATE = 16000  # Sample rate
CHUNK = 1024  # Number of frames per buffer

def is_silent(data, threshold=500):
    """Returns 'True' if below the 'silent' threshold"""
    return np.abs(np.frombuffer(data, dtype=np.int16)).max() < threshold

def record_audio():
    recognizer = sr.Recognizer()
    
    audio_data = io.BytesIO()
    p = pyaudio.PyAudio()
    
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Listening...")
    silence_start = None
    audio_chunks = []
    
    while True:
        data = stream.read(CHUNK)
        audio_chunks.append(data)
        
        if is_silent(data):
            if silence_start is None:
                silence_start = time.time()
            elif time.time() - silence_start > 3:
                break
        else:
            silence_start = None
    
    print("Recording complete")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    audio_data.write(b''.join(audio_chunks))
    audio_data.seek(0)
    return audio_data, recognizer

def speech_to_text(audio_data, recognizer):
    audio = sr.AudioData(audio_data.read(), RATE, 2)
    text = recognizer.recognize_google(audio)
    return text

In [146]:
from datetime import datetime

audio_data, recognizer = record_audio()
    
# Convert speech to text
text = speech_to_text(audio_data, recognizer)
print("Recognized Text:", text)
amt, item = extract_amount_item(text)
category = get_category(item)
timestamp = datetime.now()

    # Create a new DataFrame for the new row
new_row_df = pd.DataFrame([[amt, item, category,timestamp]], columns=['Amount', 'Item', 'Category', 'Timestamp'])

    # Concatenate the new row to the existing DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)

    # Display the DataFrame
print(df)

Listening...
Recording complete
Recognized Text: I got medicines worth Rs 300
   Amount                 Item        Category                  Timestamp
0      50              bananas          others                        NaT
1     100            Starbucks            food                        NaT
2     200     electricity bill       household                        NaT
3     150                 fare  transportation                        NaT
4     300               ticket   entertainment                        NaT
5      75                 cafe            food                        NaT
6     120             shopping        shopping                        NaT
7     500                phone   entertainment                        NaT
8      25                 book       education                        NaT
9    50.5               fruits            food                        NaT
10    100                water            food                        NaT
11    200            groceries    

In [ ]:
df.iloc[:60]